In [ ]:
import pandas as pd
import itertools
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('reduced_data.csv')

target_vars = df.columns[:2]
feature_vars = df.columns[2:]

correlation_with_target1 = df[feature_vars].apply(lambda x: x.corr(df[target_vars[0]])).abs()
correlation_with_target2 = df[feature_vars].apply(lambda x: x.corr(df[target_vars[1]])).abs()

max_correlation = correlation_with_target1.combine(correlation_with_target2, max)

top_features = max_correlation.sort_values(ascending=False).index[:15]

print(f"Top features correlated with targets: {top_features}")

plt.figure(figsize=(15, 12))  
sns.heatmap(df[top_features].corr().round(2), annot=True, cmap='coolwarm', fmt='.2f')  
plt.savefig('Correlation Matrix of Top Features_Full sample.jpeg', dpi=1000, bbox_inches='tight')
plt.show()

def create_interaction_features(df, features):
    for feature1, feature2 in itertools.combinations(features, 2):
        interaction_feature_name = f"{feature1}_x_{feature2}"
        df[interaction_feature_name] = df[feature1] * df[feature2]
    return df

df_with_interactions = create_interaction_features(df.copy(), top_features)

print(df_with_interactions.head())

df_with_interactions.to_csv('data_with_interactions.csv', index=False)


In [ ]:
#xgboost
import numpy as np
import pandas as pd
import warnings
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
import xgboost as xgb
from scipy.stats import pearsonr

try:
    data = pd.read_csv('data_with_interactions.csv')
    print("Data loaded successfully")
except FileNotFoundError:
    print("Data file not found")
    raise

X = data.iloc[:, 2:].values
y = data.iloc[:, :2].values 

def composite_metric(y_true, y_pred, alpha=0.5, beta=0.1):
    """
    Calculate the multi-objective composite evaluation metric.
    :param y_true: Array of true values, shape (n_samples, n_targets).
    :param y_pred: Array of predicted values, shape (n_samples, n_targets).
    :param alpha: Weight of the weighted root mean square error (WRMSE) in the composite evaluation metric.
    :param beta: Weight of the inter-target correlation in the composite evaluation metric.
    :return: Composite evaluation metric.
    """

    n_targets = y_true.shape[1]

    mse_list = []
    mae_list = []
    max_mse = 0
    max_mae = 0

    for i in range(n_targets):
        mse = mean_squared_error(y_true[:, i], y_pred[:, i])
        mae = mean_absolute_error(y_true[:, i], y_pred[:, i])
        mse_list.append(mse)
        mae_list.append(mae)
        if mse > max_mse:
            max_mse = mse
        if mae > max_mae:
            max_mae = mae

    normalized_mse = [mse / max_mse for mse in mse_list]
    normalized_mae = [mae / max_mae for mae in mae_list]

    inverse_mse = [1 / mse if mse != 0 else 0 for mse in normalized_mse]
    sum_inverse_mse = sum(inverse_mse)
    weights = [inv_mse / sum_inverse_mse for inv_mse in inverse_mse]

    wrmse = np.sqrt(sum([w * mse for w, mse in zip(weights, normalized_mse)]))
    wmae = sum([w * mae for w, mae in zip(weights, normalized_mae)])

    correlation_sum = 0
    for i in range(n_targets):
        for j in range(i + 1, n_targets):
            corr, _ = pearsonr(y_true[:, i], y_true[:, j])
            correlation_sum += corr * mse_list[i] * mse_list[j]

    composite_score = alpha * wrmse + (1 - alpha) * wmae + beta * correlation_sum
    return composite_score

with warnings.catch_warnings():
    warnings.simplefilter("ignore")


def train_and_evaluate(X, y, alpha=0.5, beta=0.1):
    composite_scores = []

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
    for train_index, val_index in kf.split(X_train):
        X_tr, X_val = X_train[train_index], X_train[val_index]
        y_tr, y_val = y_train[train_index], y_train[val_index]

        model1 = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
        model2 = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)

        model1.fit(X_tr, y_tr[:, 0])
        model2.fit(X_tr, y_tr[:, 1])

        y_pred1 = model1.predict(X_val)
        y_pred2 = model2.predict(X_val)

        y_val_combined = np.vstack((y_val[:, 0], y_val[:, 1])).T
        y_pred_combined = np.vstack((y_pred1, y_pred2)).T

        score = composite_metric(y_val_combined, y_pred_combined, alpha, beta)
        composite_scores.append(score)

    mean_composite_score = np.mean(composite_scores)

    model1.fit(X_train, y_train[:, 0])
    model2.fit(X_train, y_train[:, 1])

    y_pred1_test = model1.predict(X_test)
    y_pred2_test = model2.predict(X_test)

    y_test_combined = np.vstack((y_test[:, 0], y_test[:, 1])).T
    y_pred_combined_test = np.vstack((y_pred1_test, y_pred2_test)).T

    test_score = composite_metric(y_test_combined, y_pred_combined_test, alpha, beta)
    print(f"Test Composite Score: {test_score}")

    return mean_composite_score, test_score

mean_composite_score, test_composite_score = train_and_evaluate(X, y)
print(f"Mean Composite Score (Cross-validation): {mean_composite_score}")
print(f"Test Composite Score: {test_composite_score}")

In [ ]:
#adboost
import numpy as np
import pandas as pd
import warnings
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import AdaBoostRegressor
from scipy.stats import pearsonr

try:
    data = pd.read_csv('data_with_interactions.csv')
    print("Data loaded successfully")
except FileNotFoundError:
    print("Data file not found")
    raise

X = data.iloc[:, 2:].values
y = data.iloc[:, :2].values  

def composite_metric(y_true, y_pred, alpha=0.5, beta=0.1):
    """
    Calculate the multi-objective composite evaluation metric.
    :param y_true: Array of true values, shape (n_samples, n_targets).
    :param y_pred: Array of predicted values, shape (n_samples, n_targets).
    :param alpha: Weight of the weighted root mean square error (WRMSE) in the composite evaluation metric.
    :param beta: Weight of the inter-target correlation in the composite evaluation metric.
    :return: Composite evaluation metric.
    """
    n_targets = y_true.shape[1]


    mse_list = []
    mae_list = []
    max_mse = 0
    max_mae = 0

    for i in range(n_targets):
        mse = mean_squared_error(y_true[:, i], y_pred[:, i])
        mae = mean_absolute_error(y_true[:, i], y_pred[:, i])
        mse_list.append(mse)
        mae_list.append(mae)
        if mse > max_mse:
            max_mse = mse
        if mae > max_mae:
            max_mae = mae

  
    normalized_mse = [mse / max_mse for mse in mse_list]
    normalized_mae = [mae / max_mae for mae in mae_list]

  
    inverse_mse = [1 / mse if mse != 0 else 0 for mse in normalized_mse]
    sum_inverse_mse = sum(inverse_mse)
    weights = [inv_mse / sum_inverse_mse for inv_mse in inverse_mse]

 
    wrmse = np.sqrt(sum([w * mse for w, mse in zip(weights, normalized_mse)]))
    wmae = sum([w * mae for w, mae in zip(weights, normalized_mae)])

    correlation_sum = 0
    for i in range(n_targets):
        for j in range(i + 1, n_targets):
            corr, _ = pearsonr(y_true[:, i], y_true[:, j])
            correlation_sum += corr * mse_list[i] * mse_list[j]


    composite_score = alpha * wrmse + (1 - alpha) * wmae + beta * correlation_sum
    return composite_score


with warnings.catch_warnings():
    warnings.simplefilter("ignore")


def train_and_evaluate(X, y, alpha=0.5, beta=0.1):
    composite_scores = []

  
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
    for train_index, val_index in kf.split(X_train):
        X_tr, X_val = X_train[train_index], X_train[val_index]
        y_tr, y_val = y_train[train_index], y_train[val_index]

        model1 = AdaBoostRegressor(n_estimators=100, random_state=42)
        model2 = AdaBoostRegressor(n_estimators=100, random_state=42)

        model1.fit(X_tr, y_tr[:, 0])
        model2.fit(X_tr, y_tr[:, 1])

        y_pred1 = model1.predict(X_val)
        y_pred2 = model2.predict(X_val)

        y_val_combined = np.vstack((y_val[:, 0], y_val[:, 1])).T
        y_pred_combined = np.vstack((y_pred1, y_pred2)).T

        score = composite_metric(y_val_combined, y_pred_combined, alpha, beta)
        composite_scores.append(score)

    mean_composite_score = np.mean(composite_scores)


    model1.fit(X_train, y_train[:, 0])
    model2.fit(X_train, y_train[:, 1])

    y_pred1_test = model1.predict(X_test)
    y_pred2_test = model2.predict(X_test)

    y_test_combined = np.vstack((y_test[:, 0], y_test[:, 1])).T
    y_pred_combined_test = np.vstack((y_pred1_test, y_pred2_test)).T

    test_score = composite_metric(y_test_combined, y_pred_combined_test, alpha, beta)
    print(f"Test Composite Score: {test_score}")

    return mean_composite_score, test_score

mean_composite_score, test_composite_score = train_and_evaluate(X, y)
print(f"Mean Composite Score (Cross-validation): {mean_composite_score}")
print(f"Test Composite Score: {test_composite_score}")

In [ ]:
#catboost
import numpy as np
import pandas as pd
import warnings
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from catboost import CatBoostRegressor
from scipy.stats import pearsonr


try:
    data = pd.read_csv('data_with_interactions.csv')
    print("Data loaded successfully")
except FileNotFoundError:
    print("Data file not found")
    raise


X = data.iloc[:, 2:].values
y = data.iloc[:, :2].values 

def composite_metric(y_true, y_pred, alpha=0.5, beta=0.1):
    """
    Calculate the multi-objective composite evaluation metric.
    :param y_true: Array of true values, shape (n_samples, n_targets).
    :param y_pred: Array of predicted values, shape (n_samples, n_targets).
    :param alpha: Weight of the weighted root mean square error (WRMSE) in the composite evaluation metric.
    :param beta: Weight of the inter-target correlation in the composite evaluation metric.
    :return: Composite evaluation metric.
    """
    n_targets = y_true.shape[1]

    mse_list = []
    mae_list = []
    max_mse = 0
    max_mae = 0

    for i in range(n_targets):
        mse = mean_squared_error(y_true[:, i], y_pred[:, i])
        mae = mean_absolute_error(y_true[:, i], y_pred[:, i])
        mse_list.append(mse)
        mae_list.append(mae)
        if mse > max_mse:
            max_mse = mse
        if mae > max_mae:
            max_mae = mae

    normalized_mse = [mse / max_mse for mse in mse_list]
    normalized_mae = [mae / max_mae for mae in mae_list]

    inverse_mse = [1 / mse if mse != 0 else 0 for mse in normalized_mse]
    sum_inverse_mse = sum(inverse_mse)
    weights = [inv_mse / sum_inverse_mse for inv_mse in inverse_mse]

    wrmse = np.sqrt(sum([w * mse for w, mse in zip(weights, normalized_mse)]))
    wmae = sum([w * mae for w, mae in zip(weights, normalized_mae)])

    correlation_sum = 0
    for i in range(n_targets):
        for j in range(i + 1, n_targets):
            corr, _ = pearsonr(y_true[:, i], y_true[:, j])
            correlation_sum += corr * mse_list[i] * mse_list[j]

    composite_score = alpha * wrmse + (1 - alpha) * wmae + beta * correlation_sum
    return composite_score

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

def train_and_evaluate(X, y, alpha=0.5, beta=0.1):
    composite_scores = []

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
    for train_index, val_index in kf.split(X_train):
        X_tr, X_val = X_train[train_index], X_train[val_index]
        y_tr, y_val = y_train[train_index], y_train[val_index]

        model1 = CatBoostRegressor(iterations=100, random_seed=42, verbose=0)
        model2 = CatBoostRegressor(iterations=100, random_seed=42, verbose=0)

        model1.fit(X_tr, y_tr[:, 0])
        model2.fit(X_tr, y_tr[:, 1])

        y_pred1 = model1.predict(X_val)
        y_pred2 = model2.predict(X_val)

        y_val_combined = np.vstack((y_val[:, 0], y_val[:, 1])).T
        y_pred_combined = np.vstack((y_pred1, y_pred2)).T

        score = composite_metric(y_val_combined, y_pred_combined, alpha, beta)
        composite_scores.append(score)

    mean_composite_score = np.mean(composite_scores)

    model1.fit(X_train, y_train[:, 0])
    model2.fit(X_train, y_train[:, 1])

    y_pred1_test = model1.predict(X_test)
    y_pred2_test = model2.predict(X_test)

    y_test_combined = np.vstack((y_test[:, 0], y_test[:, 1])).T
    y_pred_combined_test = np.vstack((y_pred1_test, y_pred2_test)).T

    test_score = composite_metric(y_test_combined, y_pred_combined_test, alpha, beta)
    print(f"Test Composite Score: {test_score}")

    return mean_composite_score, test_score

mean_composite_score, test_composite_score = train_and_evaluate(X, y)
print(f"Mean Composite Score (Cross-validation): {mean_composite_score}")
print(f"Test Composite Score: {test_composite_score}")

In [ ]:
#GBRT
import numpy as np
import pandas as pd
import warnings
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor
from scipy.stats import pearsonr

try:
    data = pd.read_csv('data_with_interactions.csv')
    print("Data loaded successfully")
except FileNotFoundError:
    print("Data file not found")
    raise

X = data.iloc[:, 2:].values
y = data.iloc[:, :2].values  

def composite_metric(y_true, y_pred, alpha=0.5, beta=0.1):
    """
    Calculate the multi-objective composite evaluation metric.
    :param y_true: Array of true values, shape (n_samples, n_targets).
    :param y_pred: Array of predicted values, shape (n_samples, n_targets).
    :param alpha: Weight of the weighted root mean square error (WRMSE) in the composite evaluation metric.
    :param beta: Weight of the inter-target correlation in the composite evaluation metric.
    :return: Composite evaluation metric.
    """
    n_targets = y_true.shape[1]

    mse_list = []
    mae_list = []
    max_mse = 0
    max_mae = 0

    for i in range(n_targets):
        mse = mean_squared_error(y_true[:, i], y_pred[:, i])
        mae = mean_absolute_error(y_true[:, i], y_pred[:, i])
        mse_list.append(mse)
        mae_list.append(mae)
        if mse > max_mse:
            max_mse = mse
        if mae > max_mae:
            max_mae = mae

    normalized_mse = [mse / max_mse for mse in mse_list]
    normalized_mae = [mae / max_mae for mae in mae_list]

    inverse_mse = [1 / mse if mse != 0 else 0 for mse in normalized_mse]
    sum_inverse_mse = sum(inverse_mse)
    weights = [inv_mse / sum_inverse_mse for inv_mse in inverse_mse]

    wrmse = np.sqrt(sum([w * mse for w, mse in zip(weights, normalized_mse)]))
    wmae = sum([w * mae for w, mae in zip(weights, normalized_mae)])

    correlation_sum = 0
    for i in range(n_targets):
        for j in range(i + 1, n_targets):
            corr, _ = pearsonr(y_true[:, i], y_true[:, j])
            correlation_sum += corr * mse_list[i] * mse_list[j]

    composite_score = alpha * wrmse + (1 - alpha) * wmae + beta * correlation_sum
    return composite_score

with warnings.catch_warnings():
    warnings.simplefilter("ignore")

def train_and_evaluate(X, y, alpha=0.5, beta=0.1):
    composite_scores = []

   
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
    for train_index, val_index in kf.split(X_train):
        X_tr, X_val = X_train[train_index], X_train[val_index]
        y_tr, y_val = y_train[train_index], y_train[val_index]

        model1 = GradientBoostingRegressor(n_estimators=100, random_state=42)
        model2 = GradientBoostingRegressor(n_estimators=100, random_state=42)

        model1.fit(X_tr, y_tr[:, 0])
        model2.fit(X_tr, y_tr[:, 1])

        y_pred1 = model1.predict(X_val)
        y_pred2 = model2.predict(X_val)

        y_val_combined = np.vstack((y_val[:, 0], y_val[:, 1])).T
        y_pred_combined = np.vstack((y_pred1, y_pred2)).T

        score = composite_metric(y_val_combined, y_pred_combined, alpha, beta)
        composite_scores.append(score)

    mean_composite_score = np.mean(composite_scores)


    model1.fit(X_train, y_train[:, 0])
    model2.fit(X_train, y_train[:, 1])

    y_pred1_test = model1.predict(X_test)
    y_pred2_test = model2.predict(X_test)

    y_test_combined = np.vstack((y_test[:, 0], y_test[:, 1])).T
    y_pred_combined_test = np.vstack((y_pred1_test, y_pred2_test)).T

    test_score = composite_metric(y_test_combined, y_pred_combined_test, alpha, beta)
    print(f"Test Composite Score: {test_score}")

    return mean_composite_score, test_score


mean_composite_score, test_composite_score = train_and_evaluate(X, y)
print(f"Mean Composite Score (Cross-validation): {mean_composite_score}")
print(f"Test Composite Score: {test_composite_score}")


In [ ]:
#RF
import numpy as np
import pandas as pd
import warnings
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import pearsonr


try:
    data = pd.read_csv('data_with_interactions.csv')
    print("Data loaded successfully")
except FileNotFoundError:
    print("Data file not found")
    raise


X = data.iloc[:, 2:].values
y = data.iloc[:, :2].values  

def composite_metric(y_true, y_pred, alpha=0.5, beta=0.1):
    """
    Calculate the multi-objective composite evaluation metric.
    :param y_true: Array of true values, shape (n_samples, n_targets).
    :param y_pred: Array of predicted values, shape (n_samples, n_targets).
    :param alpha: Weight of the weighted root mean square error (WRMSE) in the composite evaluation metric.
    :param beta: Weight of the inter-target correlation in the composite evaluation metric.
    :return: Composite evaluation metric.
    """
    n_targets = y_true.shape[1]

   
    mse_list = []
    mae_list = []
    max_mse = 0
    max_mae = 0

    for i in range(n_targets):
        mse = mean_squared_error(y_true[:, i], y_pred[:, i])
        mae = mean_absolute_error(y_true[:, i], y_pred[:, i])
        mse_list.append(mse)
        mae_list.append(mae)
        if mse > max_mse:
            max_mse = mse
        if mae > max_mae:
            max_mae = mae

 
    normalized_mse = [mse / max_mse for mse in mse_list]
    normalized_mae = [mae / max_mae for mae in mae_list]


    inverse_mse = [1 / mse if mse != 0 else 0 for mse in normalized_mse]
    sum_inverse_mse = sum(inverse_mse)
    weights = [inv_mse / sum_inverse_mse for inv_mse in inverse_mse]

  
    wrmse = np.sqrt(sum([w * mse for w, mse in zip(weights, normalized_mse)]))
    wmae = sum([w * mae for w, mae in zip(weights, normalized_mae)])


    correlation_sum = 0
    for i in range(n_targets):
        for j in range(i + 1, n_targets):
            corr, _ = pearsonr(y_true[:, i], y_true[:, j])
            correlation_sum += corr * mse_list[i] * mse_list[j]


    composite_score = alpha * wrmse + (1 - alpha) * wmae + beta * correlation_sum
    return composite_score


with warnings.catch_warnings():
    warnings.simplefilter("ignore")


def train_and_evaluate(X, y, alpha=0.5, beta=0.1):
    composite_scores = []

   
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
 
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
    for train_index, val_index in kf.split(X_train):
        X_tr, X_val = X_train[train_index], X_train[val_index]
        y_tr, y_val = y_train[train_index], y_train[val_index]

        model1 = RandomForestRegressor(n_estimators=100, random_state=42)
        model2 = RandomForestRegressor(n_estimators=100, random_state=42)

        model1.fit(X_tr, y_tr[:, 0])
        model2.fit(X_tr, y_tr[:, 1])

        y_pred1 = model1.predict(X_val)
        y_pred2 = model2.predict(X_val)

        y_val_combined = np.vstack((y_val[:, 0], y_val[:, 1])).T
        y_pred_combined = np.vstack((y_pred1, y_pred2)).T

        score = composite_metric(y_val_combined, y_pred_combined, alpha, beta)
        composite_scores.append(score)

    mean_composite_score = np.mean(composite_scores)

 
    model1.fit(X_train, y_train[:, 0])
    model2.fit(X_train, y_train[:, 1])

    y_pred1_test = model1.predict(X_test)
    y_pred2_test = model2.predict(X_test)

    y_test_combined = np.vstack((y_test[:, 0], y_test[:, 1])).T
    y_pred_combined_test = np.vstack((y_pred1_test, y_pred2_test)).T

    test_score = composite_metric(y_test_combined, y_pred_combined_test, alpha, beta)
    print(f"Test Composite Score: {test_score}")

    return mean_composite_score, test_score


mean_composite_score, test_composite_score = train_and_evaluate(X, y)
print(f"Mean Composite Score (Cross-validation): {mean_composite_score}")
print(f"Test Composite Score: {test_composite_score}")

In [ ]:
#ExtraTree
import numpy as np
import pandas as pd
import warnings
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import ExtraTreesRegressor
from scipy.stats import pearsonr


try:
    data = pd.read_csv('data_with_interactions.csv')
    print("Data loaded successfully")
except FileNotFoundError:
    print("Data file not found")
    raise


X = data.iloc[:, 2:].values
y = data.iloc[:, :2].values  

def composite_metric(y_true, y_pred, alpha=0.5, beta=0.1):
    """
    Calculate the multi-objective composite evaluation metric.
    :param y_true: Array of true values, shape (n_samples, n_targets).
    :param y_pred: Array of predicted values, shape (n_samples, n_targets).
    :param alpha: Weight of the weighted root mean square error (WRMSE) in the composite evaluation metric.
    :param beta: Weight of the inter-target correlation in the composite evaluation metric.
    :return: Composite evaluation metric.
    """
    n_targets = y_true.shape[1]


    mse_list = []
    mae_list = []
    max_mse = 0
    max_mae = 0

    for i in range(n_targets):
        mse = mean_squared_error(y_true[:, i], y_pred[:, i])
        mae = mean_absolute_error(y_true[:, i], y_pred[:, i])
        mse_list.append(mse)
        mae_list.append(mae)
        if mse > max_mse:
            max_mse = mse
        if mae > max_mae:
            max_mae = mae


    normalized_mse = [mse / max_mse for mse in mse_list]
    normalized_mae = [mae / max_mae for mae in mae_list]


    inverse_mse = [1 / mse if mse != 0 else 0 for mse in normalized_mse]
    sum_inverse_mse = sum(inverse_mse)
    weights = [inv_mse / sum_inverse_mse for inv_mse in inverse_mse]


    wrmse = np.sqrt(sum([w * mse for w, mse in zip(weights, normalized_mse)]))
    wmae = sum([w * mae for w, mae in zip(weights, normalized_mae)])


    correlation_sum = 0
    for i in range(n_targets):
        for j in range(i + 1, n_targets):
            corr, _ = pearsonr(y_true[:, i], y_true[:, j])
            correlation_sum += corr * mse_list[i] * mse_list[j]

 
    composite_score = alpha * wrmse + (1 - alpha) * wmae + beta * correlation_sum
    return composite_score


with warnings.catch_warnings():
    warnings.simplefilter("ignore")


def train_and_evaluate(X, y, alpha=0.5, beta=0.1):
    composite_scores = []

 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    
    for train_index, val_index in kf.split(X_train):
        X_tr, X_val = X_train[train_index], X_train[val_index]
        y_tr, y_val = y_train[train_index], y_train[val_index]

        model1 = ExtraTreesRegressor(n_estimators=100, random_state=42)
        model2 = ExtraTreesRegressor(n_estimators=100, random_state=42)

        model1.fit(X_tr, y_tr[:, 0])
        model2.fit(X_tr, y_tr[:, 1])

        y_pred1 = model1.predict(X_val)
        y_pred2 = model2.predict(X_val)

        y_val_combined = np.vstack((y_val[:, 0], y_val[:, 1])).T
        y_pred_combined = np.vstack((y_pred1, y_pred2)).T

        score = composite_metric(y_val_combined, y_pred_combined, alpha, beta)
        composite_scores.append(score)

    mean_composite_score = np.mean(composite_scores)


    model1.fit(X_train, y_train[:, 0])
    model2.fit(X_train, y_train[:, 1])

    y_pred1_test = model1.predict(X_test)
    y_pred2_test = model2.predict(X_test)

    y_test_combined = np.vstack((y_test[:, 0], y_test[:, 1])).T
    y_pred_combined_test = np.vstack((y_pred1_test, y_pred2_test)).T

    test_score = composite_metric(y_test_combined, y_pred_combined_test, alpha, beta)
    print(f"Test Composite Score: {test_score}")

    return mean_composite_score, test_score


mean_composite_score, test_composite_score = train_and_evaluate(X, y)
print(f"Mean Composite Score (Cross-validation): {mean_composite_score}")
print(f"Test Composite Score: {test_composite_score}")